<a href="https://www.kaggle.com/code/cocolabhub/10-02-generativellmwithhuggingface-datasets?scriptVersionId=222079618" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## 데이터셋 관련\
- 실습 데이터(GTP 통해 생성) : https://huggingface.co/datasets/shangrilar/ko_text2sql
- 게임 데이터베이스 맞게 형성

In [1]:
!pip install transformers==4.40.1 datasets==2.19.0 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.3.1 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.


### 데이터 로딩
- 영어 text2sql : https://huggingface.co/datasets/Salesforce/wikisql
- 한글어 관련 데이터 hub : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=data&dataSetSn=71351

In [2]:
!rm -rf *
# !ls -lah

In [3]:
# huggingface dataset subset name and compress file name
# dataset_name = 'origin'  # GPT 통한 query 생성
dataset_name = 'clean'  # 정답 확인 정제 데이터

In [4]:
from datasets import load_dataset
# https://huggingface.co/datasets/shangrilar/ko_text2sql/viewer/origin
hf_dataset_text2sql = load_dataset('shangrilar/ko_text2sql', dataset_name)
hf_dataset_text2sql

Generating train split:   0%|          | 0/28753 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 28753
    })
    test: Dataset({
        features: ['db_id', 'context', 'question', 'answer'],
        num_rows: 112
    })
})

In [5]:
df_dataset_text2sql_train = hf_dataset_text2sql['train'].to_pandas()
df_dataset_text2sql_train = df_dataset_text2sql_train.dropna().sample(frac=1, random_state=42)

In [6]:
# huggingface dataset to pandas DataFrame with train
df_dataset_text2sql_train = df_dataset_text2sql_train.query('db_id != 1')  # 정상 데이터들만 추출
df_dataset_text2sql_train.head(3)

,db_id,context,question,answer
4246,2,"CREATE TABLE user_emojis (\n ""user_id"" INT ...",이모지의 ID와 구매 일자를 모두 찾아주세요,"SELECT emoji_id, purchased_at FROM user_emojis;"
14133,6,"CREATE TABLE payments (\n ""payment_id"" SERIAL...",주문번호가 200번이거나 결제일자가 2021년 5월 1일 이후인 결제 내역을 알려줘,SELECT * FROM payments WHERE order_id = 200 OR...
16291,7,"CREATE TABLE calendars (\n ""calendar_id"" SERI...",2024년 1월 이후 가격이 비싼 상위 10개의 일정을 보여줘,SELECT * FROM calendars WHERE date > '2024-01-...


### 학습 위한 prompt 명령어 완성

In [7]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

In [8]:
# 필요한 컬럼 추출해 프롬프트 명령어 만드리
from tqdm import tqdm

df_total_count = len(df_dataset_text2sql_train)
for idx, row in tqdm(df_dataset_text2sql_train.iterrows(), total=df_total_count, desc='Generating prompts') : # DataFrame row 단위 loop
    # make_prompt(ddl, question, query='')
    train_promt_command = make_prompt(ddl= row['context']
                        , question = row['question']
                        , query=row['answer'])
    df_dataset_text2sql_train.loc[idx, 'text'] = train_promt_command
    pass

df_dataset_text2sql_train.head(3)

Generating prompts: 100%|██████████| 25363/25363 [00:05<00:00, 4550.77it/s]


,db_id,context,question,answer,text
4246,2,"CREATE TABLE user_emojis (\n ""user_id"" INT ...",이모지의 ID와 구매 일자를 모두 찾아주세요,"SELECT emoji_id, purchased_at FROM user_emojis;",당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
14133,6,"CREATE TABLE payments (\n ""payment_id"" SERIAL...",주문번호가 200번이거나 결제일자가 2021년 5월 1일 이후인 결제 내역을 알려줘,SELECT * FROM payments WHERE order_id = 200 OR...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...
16291,7,"CREATE TABLE calendars (\n ""calendar_id"" SERI...",2024년 1월 이후 가격이 비싼 상위 10개의 일정을 보여줘,SELECT * FROM calendars WHERE date > '2024-01-...,당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question...


In [9]:
!mkdir ./datasets

### Test 위한 prompt 명령어 완성

In [10]:
# 재사용 위한 저장
dataset_path = 'datasets'
df_dataset_text2sql_train.to_csv(f'{dataset_path}/train.csv')

In [11]:
# huggingface dataset to pandas DataFrame with test
# df_dataset_text2sql_test = hf_dataset_text2sql['test'].to_pandas()
# df_dataset_text2sql_test.head(3)

In [12]:
# 필요한 컬럼 추출해 프롬프트 명령어 만드리
# from tqdm import tqdm

# df_total_count = len(df_dataset_text2sql_test)
# for idx, row in tqdm(df_dataset_text2sql_test.iterrows()
#                      , total=df_total_count, desc='Generating prompts') : # DataFrame row 단위 loop
#     # make_prompt(ddl, question, query='')
#     train_promt_command = make_prompt(ddl= row['context']
#                         , question = row['question']
#                         , query='')
#     df_dataset_text2sql_test.loc[idx, 'prompt'] = train_promt_command
#     pass

# df_dataset_text2sql_test.head(3)

In [13]:
# df_dataset_text2sql_test.to_csv(f'{dataset_path}/test.csv')

## linux 명령어 알기

In [14]:
!ls -la
# !ls -la ../
# !pwd

total 56
drwxr-xr-x 3 root root  4096 Feb 12 01:39 .
drwxr-xr-x 6 root root  4096 Feb 12 01:38 ..
drwxr-xr-x 2 root root  4096 Feb 12 01:39 datasets
-rw-r--r-- 1 root root 41952 Feb 12 01:39 __notebook__.ipynb


In [15]:
!zip -r ko_text2sql_{dataset_name}_Datasets.zip *

  adding: datasets/ (stored 0%)
  adding: datasets/train.csv (deflated 90%)
  adding: __notebook__.ipynb (deflated 85%)
